# What is Stabilizer Formalism and why do I need it? 

Quantum error correction is an important domain in the universe of quantum computing. There are several quantum error correction codes to mitigate and combat the errors. The **stabilizer formalism** is a framework for the quantum error correction codes, please observe that it is not a new error correction code on its own by rather a new representation to work with. There are also many error correction codes which doesn't fall under the umberalla of stabilizer formalism. We will not discuss those in this article.

Simplest example of error correcting code is the repeatition code. Before we dive deep into the stabilizer formalism, let us draw some inspiration by examining the repeation code from a new angle. 

<!-- The **stabilizer formalism** is a framework used in quantum error correction. It enables the characterization of quantum states using stabilizer groups, which are subgroups of the Pauli group. This notebook demonstrates the stabilizer formalism through examples implemented in Python with PennyLane. -->

## Repeatition code

As you know, the codespace of the repeatition code is defined by,

$$ \ket{\psi} = \alpha \ket{000} + \beta \ket{111} $$

Imagine if there is a bit flip error an error on the codespace it leads to an error space given by,

$$ X_2 \ket{\psi}_e = \alpha \ket{010} + \beta \ket{101}  $$

How is this error detected? Unlike classical case, no-cloning theorem prevents us from measuring the qubits to learn about the error. Before, worrying about how to detect an error, first let us understand what kinds of errors are possible. Observe the fact that the only detectable error is a single bit flip error on any of the qubits, therefore all we need to check is if any ONE of the bits flipped. In order to do this, we can build a circuit which checks if the first and second qubits are same or different and similary if second and third qubits are same or different. Okay how do we build one? Let us use PennyLane! 

It is divided into 3 steps. First step is to prepare the state, as from above we can use any alpha and beta to create the state but let us use the simplest one, that is using 1/sqrt 2 for both alpha and beta. The code block below demonstrates that


In [3]:
import pennylane as qml
from pennylane import numpy as np

# Define the device
dev = qml.device("default.qubit", wires=3)

# Define the quantum circuit for encoding the state
@qml.qnode(dev)
def encode_circuit():
    # Apply the Hadamard gate to qubit 0 (creating a superposition between |0> and |1>)
    qml.Hadamard(wires=0)
    
    # Encode the state into 3 qubits (using repetition code)
    qml.CNOT(wires=[0, 1])  # CNOT from qubit 0 to qubit 1
    qml.CNOT(wires=[0, 2])  # CNOT from qubit 0 to qubit 2
    
    return qml.state()  # Return the full quantum state

# Run the encoding circuit
encoded_state = encode_circuit()
print("Encoded State:", encoded_state)


Encoded State: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


$$
\text{The output vector represents the quantum state } \frac{1}{\sqrt{2}}(|000\rangle + |111\rangle), 
$$
$$
\text{where the amplitudes for } |000\rangle \text{ and } |111\rangle \text{ are } \frac{1}{\sqrt{2}},
$$
$$
\text{and the amplitudes for all other states are 0.}
$$
Now let us introduce the X error:

In [ ]:
# Define the quantum circuit to introduce X error on qubit 2
@qml.qnode(dev)
def apply_x_error():
    # Apply X error on qubit 2 (flip the state of qubit 2)
    qml.PauliX(wires=2)
    
    return qml.state()  # Return the state after the error is applied

# Run the circuit with the X error on qubit 2
state_with_error = apply_x_error()
print("State with X error on qubit 2:", state_with_error)


Now let us build the circuit to detect the error.

In [5]:
# Define the quantum circuit to detect the error on the qubits
@qml.qnode(dev)
def detect_error():
    # Decode the state (repetition code)
    qml.CNOT(wires=[1, 0])  # CNOT from qubit 1 to qubit 0
    qml.CNOT(wires=[2, 0])  # CNOT from qubit 2 to qubit 0
    
    # Measure the state of all qubits
    return qml.probs(wires=[0, 1, 2])  # Probabilities of all qubits' states

# Run the circuit to detect error
probabilities = detect_error()
print("Measurement probabilities for all qubits:", probabilities)


Measurement probabilities for all qubits: [1. 0. 0. 0. 0. 0. 0. 0.]


Technically, what we did in the detection part is called the parity check. It can be done by just measuring the the operators z1z2 and z2z3. It is preciesely these opertors that are called stabilizers. What happens is that these operators break the codespace into several subspaces as shown in figure...

Now the question is that given these operators, will they form a valid quantum error correcting code? We came half way starting from repittion code to stabilizers, now let us complete the discussion by starting form the other half.

As a small exercise convince yourself that a phase flip error cannot be detected using the above code. 

# Stabilzer formalism

A few definitions first:

Pauli group:

Stabilizer group:


### Pauli Group: The Building Blocks of Quantum Operators

The **Pauli group** plays a fundamental role in quantum computing, describing essential operations on quantum bits (qubits). It consists of matrices that represent quantum transformations, with elements that either **preserve** or **flip** quantum states.

#### 🧩 Single-Qubit Pauli Group: The Elementary Set

For a single qubit, the **Pauli group** \( \mathcal{P}_1 \) is defined as the set of all Pauli matrices (plus identity), including scalar factors \( \pm1 \) and \( \pm i \):

$$
\mathcal{P}_1 = \{\pm I, \pm X, \pm Y, \pm Z, \pm iI, \pm iX, \pm iY, \pm iZ\}
$$

where the matrices are:

$$
I = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}, \quad
X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}, \quad
Y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}, \quad
Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}.
$$

Each matrix represents a fundamental operation:
- \(X\) swaps the qubit states \( |0\rangle \) and \( |1\rangle \) (**bit-flip**).
- \(Z\) introduces a phase difference between \( |0\rangle \) and \( |1\rangle \) (**phase-flip**).
- \(Y\) combines both bit-flip and phase-flip.
- \(I\) is the identity, doing nothing to the qubit.

#### 🔗 Multi-Qubit Pauli Group: Tensor Products in Action

When we deal with **multiple qubits**, the **\(n\)-qubit Pauli group** \( \mathcal{P}_n \) is constructed by **tensoring** (taking Kronecker products of) single-qubit Pauli matrices.

$$
\mathcal{P}_n = \{ \pm P_1 \otimes P_2 \otimes \dots \otimes P_n \mid P_i \in \{I, X, Y, Z\}, \text{ with scalar } \pm 1, \pm i \}.
$$

Each element in \( \mathcal{P}_n \) is an \(n\)-qubit operator built from the **individual Pauli matrices** acting on separate qubits. For example, in a **two-qubit system**:

$$
X \otimes I =
\begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} \otimes
\begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix} =
\begin{bmatrix}
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{bmatrix}.
$$

This means that \(X \otimes I\) applies the \(X\) operation **only to the first qubit**, while leaving the second qubit unchanged.

Similarly, a three-qubit operator like \(Z \otimes X \otimes Y\) applies:
- \(Z\) to the first qubit,
- \(X\) to the second qubit,
- \(Y\) to the third qubit.

#### ⚡ Key Properties of the Pauli Group

- Any two Pauli matrices either **commute** or **anti-commute**:  
  $$
  P_i P_j = \pm P_j P_i.
  $$
- The square of each Pauli matrix is the **identity**:  
  $$
  P^2 = I.
  $$
- The **eigenvalues** of any Pauli matrix are always \( \pm 1 \).

The Pauli group forms the backbone of quantum error correction, quantum gates, and stabilizer codes. By understanding how these matrices combine and interact, we gain deeper insight into quantum information processing! 🚀


### 🔗 Stabilizers in Quantum Computing

The **stabilizer formalism** is a powerful framework used in quantum error correction and quantum computing. It describes quantum states as the **common eigenstates** of commuting Pauli operators.

---

## 1️⃣ Stabilizer Group Definition  

A **stabilizer group** \( S \) is an **Abelian** subgroup of the \( n \)-qubit Pauli group \( \mathcal{P}_n \), meaning that all elements commute:

$$
g_i g_j = g_j g_i, \quad \forall g_i, g_j \in S.
$$

Additionally, \( S \) must not include \( -I^{\otimes n} \), ensuring a well-defined eigenspace.

Each stabilizer \( g_i \) satisfies:

$$
g_i |\psi\rangle = |\psi\rangle, \quad \forall g_i \in S.
$$

This means that the **stabilizer state** \( |\psi\rangle \) remains unchanged under the action of any \( g_i \).

---

## 2️⃣ Stabilizer States  

A quantum state \( |\psi\rangle \) is a **stabilizer state** if there exists a stabilizer group \( S \) with \( k \) independent generators \( g_1, g_2, \dots, g_k \), such that:

$$
S = \langle g_1, g_2, \dots, g_k \rangle.
$$

Each generator satisfies:

$$
g_i^2 = I.
$$

Since the eigenvalues of each \( g_i \) are \( \pm1 \), the stabilizer state is the **simultaneous** \( +1 \) eigenstate of all stabilizer generators.

For an \( n \)-qubit system with \( k \) independent stabilizer generators, the dimension of the stabilized subspace is:

$$
\text{Dim}(\mathcal{H}) = 2^{n-k}.
$$

Thus, stabilizer codes encode \( n-k \) logical qubits in an \( n \)-qubit system.

---

## 3️⃣ Example: Bell State as a Stabilizer State  

Consider the **Bell state**:

$$
|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}.
$$

It is stabilized by the following two commuting Pauli operators:

$$
g_1 = X \otimes X, \quad g_2 = Z \otimes Z.
$$

Both satisfy:

$$
g_1 |\Phi^+\rangle = |\Phi^+\rangle, \quad g_2 |\Phi^+\rangle = |\Phi^+\rangle.
$$

Since these two generators fully define the stabilizer group, the state \( |\Phi^+\rangle \) is uniquely determined by them.

---

## 4️⃣ Stabilizer Measurements  

Measuring a stabilizer \( g_i \) in a quantum circuit collapses the state into an eigenspace of \( g_i \), with eigenvalue \( \pm 1 \). If the measurement outcome is \( -1 \), the state is projected onto the \( -1 \) eigenspace, which can be corrected by applying \( g_i \).

---

## 5️⃣ Applications of Stabilizers  

- **Quantum Error Correction**: Stabilizer codes, such as the **Steane code** and **Surface code**, use stabilizers to detect and correct errors.  
- **Efficient Simulation**: The **Gottesman-Knill theorem** states that stabilizer circuits (Clifford circuits) can be efficiently simulated on classical computers.  
- **Fault-Tolerant Quantum Computing**: Logical qubits in stabilizer codes protect quantum information from decoherence.  

---

The **stabilizer formalism** provides a structured approach to quantum information


In [5]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Create a device with 5 qubits (3 data + 2 ancilla)
dev = qml.device("default.qubit", wires=5)

@qml.qnode(dev)
def error_detection_circuit():
    # First stabilizer: Z ⊗ Z ⊗ I
    qml.CNOT(wires=[0, 3])  # CNOT from q0 to ancilla 3
    qml.CNOT(wires=[1, 3])  # CNOT from q1 to ancilla 3

    # Second stabilizer: I ⊗ Z ⊗ Z
    qml.CNOT(wires=[1, 4])  # CNOT from q1 to ancilla 4
    qml.CNOT(wires=[2, 4])  # CNOT from q2 to ancilla 4

    # Measure ancilla qubits
    return [qml.measure(3), qml.measure(4)]

# Draw the circuit
fig, ax = qml.draw_mpl(error_detection_circuit)()
plt.show()



QuantumFunctionError: A quantum function must return either a single measurement, or a nonempty sequence of measurements.